In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier



In [ ]:
# build function for the Keras' scikit-learn API
def create_keras_model():
    """
    This function compiles and returns a Keras model.
    Should be passed to KerasClassifier in the Keras scikit-learn API.
    """

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return model

In [ ]:
# create the classifier
classifier = KerasClassifier(create_keras_model)

In [ ]:
import numpy as np
from keras.datasets import mnist

# read training data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# assemble initial data
n_initial = 1000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)[:5000]
y_pool = np.delete(y_train, initial_idx, axis=0)[:5000]

In [ ]:
pip install modAL

In [ ]:
from modAL.models import ActiveLearner

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=X_initial, y_training=y_initial,
    verbose=1
)

32/32 [==============================] - 4s 88ms/step - loss: 2.3030 - accuracy: 0.0815


In [ ]:
# the active learning loop
n_queries = 10
for idx in range(n_queries):
    print('Query no. %d' % (idx + 1))
    query_idx, query_instance = learner.query(X_pool, n_instances=100, verbose=0)
    learner.teach(
        X=X_pool[query_idx], y=y_pool[query_idx], only_new=True,
        verbose=1
    )
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)

# the final accuracy score
print(learner.score(X_test, y_test, verbose=1))

Query no. 1
4/4 [==============================] - 1s 64ms/step - loss: 2.3030 - accuracy: 0.1082
Query no. 2
4/4 [==============================] - 1s 68ms/step - loss: 2.3184 - accuracy: 0.0677
Query no. 3
4/4 [==============================] - 1s 68ms/step - loss: 2.2581 - accuracy: 0.1786
Query no. 4
4/4 [==============================] - 1s 65ms/step - loss: 2.3492 - accuracy: 0.0276
Query no. 5
4/4 [==============================] - 1s 66ms/step - loss: 2.3302 - accuracy: 0.0276
Query no. 6
4/4 [==============================] - 1s 65ms/step - loss: 2.3183 - accuracy: 0.0798
Query no. 7
4/4 [==============================] - 1s 69ms/step - loss: 2.2527 - accuracy: 0.0930
Query no. 8
4/4 [==============================] - 1s 68ms/step - loss: 2.3342 - accuracy: 0.0460
Query no. 9
4/4 [==============================] - 1s 67ms/step - loss: 2.2810 - accuracy: 0.1074
Query no. 10
 13/313 [>.............................] - ETA: 6s - loss: 2.3012 - accuracy: 0.0913